## **CONVOLUTIONAL NEURAL NETWORK --> MULTICLASS CLASSIFICATION**

This notebook consists of training a convolutional neural network with three classes of images:

- Affenpinscher dogs

- Akita dogs

- Corgi dogs

The neural network, once trained, will be able to classify these types of dog breeds.

### Packages

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
os.getcwd()

'/workspaces/Deep_Learning'

### Image Preprocessing

In [18]:
transform = transforms.Compose(
    [transforms.Resize((50,50)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

batch_size = 4
trainset = torchvision.datasets.ImageFolder(root='060_CNN_ImageClassification/MulticlassClassification/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.ImageFolder(root='060_CNN_ImageClassification/MulticlassClassification/test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

### Neural Network Setup

In [19]:
CLASSES = ['affenpinscher', 'akita', 'corgi']
NUM_CLASSES = len(CLASSES)
class ImageMulticlassClassificationNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)  
        self.pool = nn.MaxPool2d(2, 2)  
        self.conv2 = nn.Conv2d(6, 16, 3) 
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 11 * 11, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, NUM_CLASSES)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

# input = torch.rand(1, 1, 50, 50) # BS, C, H, W
model = ImageMulticlassClassificationNet()

In [20]:
loss_fn = nn.CrossEntropyLoss() #is commonly used in multi-class classification problems
optimizer = torch.optim.Adam(model.parameters()) #updates the weights of the model during training

- Training

In [21]:
NUM_EPOCHS = 10
for epoch in range(NUM_EPOCHS):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)

        
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch}/{NUM_EPOCHS}, Loss: {loss.item():.4f}')

Epoch 0/10, Loss: 1.0497
Epoch 1/10, Loss: 0.0576
Epoch 2/10, Loss: 0.3673
Epoch 3/10, Loss: 0.1208
Epoch 4/10, Loss: 0.0377
Epoch 5/10, Loss: 0.0264
Epoch 6/10, Loss: 0.0036
Epoch 7/10, Loss: 0.0092
Epoch 8/10, Loss: 0.0002
Epoch 9/10, Loss: 0.0006


- Testing

In [22]:
y_test = []
y_test_pred = []
for i, data in enumerate(testloader, 0):
    inputs, y_test_temp = data
    with torch.no_grad():
        y_test_hat_temp = model(inputs).round()
    
    y_test.extend(y_test_temp.numpy())
    y_test_pred.extend(y_test_hat_temp.numpy())

- Accuracy

In [23]:
acc = accuracy_score(y_test, np.argmax(y_test_pred, axis=1))
print(f'Accuracy: {acc*100:.2f} %')

Accuracy: 96.67 %


- Confusion Matrix

In [24]:
confusion_matrix(y_test, np.argmax(y_test_pred, axis=1))

array([[20,  0,  0],
       [ 0, 20,  0],
       [ 2,  0, 18]])

There is a small mismatch in detecting the corgi breed, as in this case it has failed and predicted as affenpinscher

--------